In [1]:
import cv2
import numpy as np
import pandas as pd
import os
import csv
import glob
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
t7sheild = os.path.join('/Volumes/T7 Shield/fox')
subj_folders = glob.glob(os.path.join('/Volumes/T7 Shield/fox/et','sub-[0-9][0-9][0-9]_phone-[a|b]-*'))
subj_folders = [folder for folder in subj_folders if 'control' not in folder]
subj_folders.sort()
subj_folders

['/Volumes/T7 Shield/fox/et/sub-095_phone-a-87a1bd3d',
 '/Volumes/T7 Shield/fox/et/sub-096_phone-b-3611b179',
 '/Volumes/T7 Shield/fox/et/sub-097_phone-b-ebeeac21',
 '/Volumes/T7 Shield/fox/et/sub-098_phone-b-29fbdee7',
 '/Volumes/T7 Shield/fox/et/sub-099_phone-b-55ff7c53',
 '/Volumes/T7 Shield/fox/et/sub-100_phone-b-66bd4dcc',
 '/Volumes/T7 Shield/fox/et/sub-104_phone-b-4cafddda',
 '/Volumes/T7 Shield/fox/et/sub-105_phone-a-448ad877',
 '/Volumes/T7 Shield/fox/et/sub-106_phone-b-1d313c9f',
 '/Volumes/T7 Shield/fox/et/sub-107_phone-a-6661b0ea',
 '/Volumes/T7 Shield/fox/et/sub-109_phone-a-463e65ac',
 '/Volumes/T7 Shield/fox/et/sub-110_phone-b-a96e2678',
 '/Volumes/T7 Shield/fox/et/sub-111_phone-a-f197f0ab',
 '/Volumes/T7 Shield/fox/et/sub-115_phone-a-f60825c4',
 '/Volumes/T7 Shield/fox/et/sub-116_phone-a-d9c05810',
 '/Volumes/T7 Shield/fox/et/sub-117_phone-b-4d98cbb8',
 '/Volumes/T7 Shield/fox/et/sub-119_phone-b-b2c5cdc2',
 '/Volumes/T7 Shield/fox/et/sub-158_phone-a-0ef92d52',
 '/Volumes

# Video Detection, Gaze Preprocessing

In [ ]:
video = os.path.join('data', 'sub-095_phone-a-87a1bd3d', 'a120071a_0.0-666.428.mp4')
gaze_data = os.path.join('data', 'sub-095_phone-a-87a1bd3d', 'gaze.csv')
world_timestamps = os.path.join('data', 'sub-095_phone-a-87a1bd3d', 'world_timestamps.csv')

print('Video: ',os.path.isfile(video))
print('Gaze Data: ',os.path.isfile(gaze_data))
print('World Timestamp Data: ',os.path.isfile(world_timestamps))

In [ ]:
# attempt 8 # would be cool to have a print out of what % complete it is...idk about that rn (not round here partner, not round here)
def detect_and_save_smartphones(video_path, output_folder, skip_frames=5):
    net = cv2.dnn.readNet("/Users/ebeard/Documents/darknet/yolov3.weights", "/Users/ebeard/Documents/darknet/cfg/yolov3.cfg")
    layer_names = net.getLayerNames()
    classes = []
    with open("/Users/ebeard/Documents/darknet/data/coco.names", "r") as f:
        classes = [line.strip() for line in f.readlines()]
    
    # Handling output layers based on the format of getUnconnectedOutLayers
    unconnected_out_layers = net.getUnconnectedOutLayers()
    if unconnected_out_layers.ndim == 1:
        output_layers = [layer_names[i - 1] for i in unconnected_out_layers.flatten()]
    else:
        output_layers = [layer_names[i[0] - 1] for i in unconnected_out_layers]

    cap = cv2.VideoCapture(video_path)
    frame_index = 0
    aoi_data = []

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    if not os.path.exists(os.path.join(output_folder, "frames")):
        os.makedirs(os.path.join(output_folder, "frames"))

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        if frame_index % skip_frames == 0:  # Process this frame
            height, width, channels = frame.shape
            blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
            net.setInput(blob)
            outs = net.forward(output_layers)

            class_ids = []
            confidences = []
            boxes = []

            for out in outs:
                for detection in out:
                    scores = detection[5:]
                    class_id = np.argmax(scores)
                    confidence = scores[class_id]
                    if confidence > 0.5:
                        center_x = int(detection[0] * width)
                        center_y = int(detection[1] * height)
                        w = int(detection[2] * width)
                        h = int(detection[3] * height)
                        x = int(center_x - w / 2)
                        y = int(center_y - h / 2)

                        if classes[class_id].lower() == 'cell phone': 
                            boxes.append([x, y, w, h])
                            confidences.append(float(confidence))
                            class_ids.append(class_id)

            indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
            found_phone = False
            largest_box = None
            max_area = 0
            
            if len(indexes) > 0:  # Check if indexes is not empty
                for i in indexes.flatten():
                    x, y, w, h = boxes[i]
                    area = w * h
                    if area > max_area:  # Find the largest bounding box
                        largest_box = (x, y, w, h)
                        max_area = area

                if largest_box is not None:
                    x, y, w, h = largest_box
                    label = str(classes[class_ids[i]])
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    cv2.putText(frame, label, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                    aoi_data.append({'frame': frame_index, 'x': x, 'y': y, 'width': w, 'height': h})
                    found_phone = True
                    
            if found_phone:
                cv2.imwrite(os.path.join(output_folder, "frames", f"frame_{frame_index}.jpg"), frame)

        frame_index += 1

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    for i in range(2):
        cv2.waitKey(1)

    # Save rectangle data to a CSV file
    with open(os.path.join(output_folder, 'smartphone_rectangle_data.csv'), 'w', newline='') as csvfile: 
        fieldnames = ['frame', 'x', 'y', 'width', 'height']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for data in aoi_data:
            writer.writerow(data)

# Example usage
#aois = detect_and_save_smartphones(video, 'full_frame', skip_frames=100)
#television = detect_and_save_smartphones(video_path = os.path.join('data', 'sub-019_tv-lego-42b0cb7d', '09e25ad2_0.0-784.762.mp4'), output_folder='full_frame_tv', skip_frames=200)


In [ ]:
def average_gaze_data(gaze_data_path, world_timestamps_path, output_folder):

    # Load gaze data
    gaze_data = pd.read_csv(gaze_data_path)

    #Load world timestamps
    world_timestamps = pd.read_csv(world_timestamps_path)

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Calculate average frame duration in nanoseconds
    frame_durations = world_timestamps['timestamp [ns]'].diff().dropna()
    average_frame_duration = frame_durations.mean()

    # Calculate the index of the world timestamp for each gaze point
    gaze_data['world_timestamp_index'] = ((gaze_data['timestamp [ns]'] - world_timestamps['timestamp [ns]'].iloc[0]) / average_frame_duration).astype(int)
    gaze_data['world_timestamp_index'] = gaze_data['world_timestamp_index'].clip(0, len(world_timestamps) - 1)  # Prevent out of bounds indexing

    # Map the calculated index to the actual world timestamps
    gaze_data['world_timestamp [ns]'] = gaze_data['world_timestamp_index'].map(world_timestamps['timestamp [ns]'])

    # Group by the world timestamp and average the gaze coordinates
    averaged_gaze = gaze_data.groupby('world_timestamp [ns]').agg({
        'gaze x [px]': 'mean',
        'gaze y [px]': 'mean'
    }).reset_index()

    averaged_gaze.to_csv(os.path.join(output_folder,'smartphone_averaged_gaze_data.csv'))
    #return averaged_gaze

# Test the function with the previously loaded gaze_data and world_timestamps
#averaged_gaze_output = average_gaze_data(gaze_data, world_timestamps)



In [ ]:
def process_aoi_data(aois_csv_path, timestamps_csv_path, output_folder, pos_diff_threshold=0.05, rolling_frame_count=3):
    # Load the AOIs data
    aois_df = pd.read_csv(aois_csv_path)

    # Load the frame timestamps data
    timestamps_df = pd.read_csv(timestamps_csv_path)

    # Assume the world timestamp data frames are indexed properly as frame numbers
    timestamps_df['frame'] = timestamps_df.index + 1  # Sequential frame numbers starting from 1

    # Map frames from AOIs to timestamps using nearest match (simplified merge without tolerance)
    merged_df = pd.merge_asof(timestamps_df.sort_values('frame'), 
                              aois_df.sort_values('frame'),
                              on='frame', 
                              direction='nearest')

    # Add previous frame AOI info for comparison
    merged_df['prev_x'] = merged_df['x'].shift(1)
    merged_df['prev_y'] = merged_df['y'].shift(1)

    # Calculate percentage differences for position only
    merged_df['diff_x'] = (merged_df['x'] - merged_df['prev_x']).abs() / merged_df['prev_x']
    merged_df['diff_y'] = (merged_df['y'] - merged_df['prev_y']).abs() / merged_df['prev_y']

    # Identify rows that exceed position difference threshold
    condition = (merged_df['diff_x'] > pos_diff_threshold) | (merged_df['diff_y'] > pos_diff_threshold)

    # Apply rolling average for the identified frames based on the specified number of frames
    for index, row in merged_df[condition].iterrows():
        window_start = max(index - (rolling_frame_count // 2), 0)
        window_end = min(index + (rolling_frame_count // 2), len(merged_df) - 1)
        window = merged_df.loc[window_start:window_end]
        merged_df.loc[index, ['x', 'y']] = window[['x', 'y']].mean()

    # Clean up the DataFrame for final output
    final_df = merged_df[['frame', 'x', 'y', 'width', 'height']]

    # Save the cleaned data to a new CSV file
    final_df.to_csv(os.path.join(output_folder, 'smartphone_processed_aoi_for_each_frame.csv'), index=False)

    print(f"Processed AOI data saved to {output_folder}")

# Usage example
'''
process_aoi_data(
    aois_csv_path='smartphone_rectangle_data.csv',
    timestamps_csv_path=world_timestamps,
    output_csv_path='smartphone_processed_aoi_for_each_frame.csv',
    pos_diff_threshold=0.05,
    rolling_frame_count=3
)'''


In [ ]:
def process_video(video_path, gaze_data_path, rectangle_data_path, output_folder, scale_factor=1.0):
    try:
        # Load data
        gaze_data = pd.read_csv(gaze_data_path)
        rectangles = pd.read_csv(rectangle_data_path)
        
        # Initialize 'within_aoi' column in gaze data
        gaze_data['within_aoi'] = False

        # Assume the world timestamp data frames are indexed properly as frame numbers
        gaze_data['frame_index'] = gaze_data.index + 1  # Sequential frame numbers starting from 1
        
        # Open the video
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print("Error opening video file")
            return
        
        # Prepare the video writer to save output
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(os.path.join(output_folder, "world__video_with_gaze_and_aoi.mp4"), fourcc, cap.get(cv2.CAP_PROP_FPS),
                              (int(cap.get(3)), int(cap.get(4))))
        
        frame_index = 0
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Get AOI for the current frame
            if frame_index in rectangles['frame'].values:
                rect = rectangles[rectangles['frame'] == frame_index].iloc[0]
                cx, cy = int(rect['x']) + int(rect['width']) // 2, int(rect['y']) + int(rect['height']) // 2
                nw, nh = int(rect['width'] * scale_factor), int(rect['height'] * scale_factor)
                nx, ny = cx - nw // 2, cy - nh // 2
                # Draw scaled rectangle (AOI)
                if np.all(np.isfinite([nx, ny, nw, nh])):
                    cv2.rectangle(frame, (nx, ny), (nx + nw, ny + nh), (0, 255, 0), 2)
            else:
                nx, ny, nw, nh = 0, 0, 0, 0  # No AOI for this frame

            # Check gaze points for this frame
            if frame_index in gaze_data['frame_index'].values:
                gaze_points = gaze_data[gaze_data['frame_index'] == frame_index]
                for index, gaze in gaze_points.iterrows():
                    gaze_x, gaze_y = int(gaze['gaze x [px]']), int(gaze['gaze y [px]'])
                    # Check if gaze is within the scaled rectangle
                    if nx <= gaze_x <= nx + nw and ny <= gaze_y <= ny + nh:
                        gaze_data.at[index, 'within_aoi'] = True  # Update gaze data
                        cv2.circle(frame, (gaze_x, gaze_y), 5, (0, 0, 255), -1)  # Red circle for gaze within AOI
                    else:
                        cv2.circle(frame, (gaze_x, gaze_y), 5, (255, 0, 0), -1)  # Blue circle for gaze outside AOI

            # Write the frame with overlays to the output video
            out.write(frame)
            
            frame_index += 1  # Increment frame index

        cap.release()
        out.release()
        cv2.destroyAllWindows()

        # Save the updated gaze data to a new CSV file
        gaze_data.to_csv(os.path.join(output_folder, 'updated_gaze_data.csv'), index=False)

        for i in range(2):
            cv2.waitKey(1)

    except Exception as e:
        print(f"An error occurred: {e}")
        # Ensure resources are released if an error occurs
        cap.release()
        out.release()
        cv2.destroyAllWindows()

        for i in range(2):
            cv2.waitKey(1)

# Example of how to call the function
#gaze_path = 'smartphone_averaged_gaze_data.csv'
#rectangle_path = 'smartphone_processed_aoi_for_each_frame.csv'
#output_video = 'smartphone-output_video_with_gaze_and_aoi.mp4'

#process_video(video, gaze_path, rectangle_path, output_video)


In [ ]:
def process_gaze_data(gaze_data_path, rectangle_data_path, video_path, output_folder, scale_factor=1.0):
    try:
        # Load data
        gaze_data = pd.read_csv(gaze_data_path)
        rectangles = pd.read_csv(rectangle_data_path)
        
        # Initialize 'within_aoi' column in gaze data
        gaze_data['within_aoi'] = False

        # Get video resolution
        video = cv2.VideoCapture(video_path)
        if not video.isOpened():
            raise ValueError("Error opening video file")
        
        video_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
        video_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
        video.release()

        # Assume the world timestamp data frames are indexed properly as frame numbers
        gaze_data['frame_index'] = gaze_data.index + 1  # Sequential frame numbers starting from 1
        
        frame_index = 0
        while frame_index < len(gaze_data):
            # Get AOI for the current frame
            if frame_index in rectangles['frame'].values:
                rect = rectangles[rectangles['frame'] == frame_index].iloc[0]
                cx, cy = int(rect['x']) + int(rect['width']) // 2, int(rect['y']) + int(rect['height']) // 2
                nw, nh = int(rect['width'] * scale_factor), int(rect['height'] * scale_factor)
                nx, ny = cx - nw // 2, cy - nh // 2
            else:
                nx, ny, nw, nh = 0, 0, 0, 0  # No AOI for this frame

            # Check gaze points for this frame
            if frame_index in gaze_data['frame_index'].values:
                gaze_points = gaze_data[gaze_data['frame_index'] == frame_index]
                for index, gaze in gaze_points.iterrows():
                    gaze_x, gaze_y = int(gaze['gaze x [px]']), int(gaze['gaze y [px]'])
                    # Scale gaze points to video resolution
                    scaled_gaze_x = int(gaze_x * video_width / gaze_data['gaze x [px]'].max())
                    scaled_gaze_y = int(gaze_y * video_height / gaze_data['gaze y [px]'].max())
                    # Check if gaze is within the scaled rectangle
                    if nx <= scaled_gaze_x <= nx + nw and ny <= scaled_gaze_y <= ny + nh:
                        gaze_data.at[index, 'within_aoi'] = True  # Update gaze data

            frame_index += 1  # Increment frame index

        # Save the updated gaze data to a new CSV file
        gaze_data.to_csv(os.path.join(output_folder, 'updated_gaze_data.csv'), index=False)

    except Exception as e:
        print(f"An error occurred: {e}")

# Example of how to call the function
#gaze_path = 'smartphone_averaged_gaze_data.csv'
#rectangle_path = 'smartphone_processed_aoi_for_each_frame.csv'
#video_path = 'path_to_video.mp4'
#output_folder = 'output_directory'

#process_gaze_data(gaze_path, rectangle_path, video_path, output_folder)

In [ ]:
def log_error(subj, error_message):
    with open("error_log.txt", "a") as log_file:
        log_file.write(f"Subject: {subj}\nError: {error_message}\n\n")

## Subj Loop

In [ ]:
for subj in subj_folders:
    print('Looking for smartphone in ', subj.split('/')[5].split('_')[0])

    subj_video = glob.glob(os.path.join(subj, '*.mp4'))[0]
    output_folder = os.path.join(subj, "smartphone_aois")
    gaze_data = os.path.join(subj, 'gaze.csv')
    world_timestamps_data = os.path.join(subj, 'world_timestamps.csv')

    if os.path.isfile(os.path.join(output_folder,"world__video_with_gaze_and_aoi.mp4")):
        print('smartphone already found!')
        continue
    
    if not os.path.isfile(subj_video):
        break
    
    try:
        detect_and_save_smartphones(subj_video, output_folder, skip_frames=100)
    except Exception as e:
        error_message = f"An error occurred: {e}"
        print(error_message)
        log_error(subj, error_message)

    if not os.path.isdir(output_folder):
        break
    
    rectangle_data = os.path.join(output_folder, 'smartphone_rectangle_data.csv')
    if not os.path.isfile(rectangle_data):
        break

    try:
        average_gaze_data(gaze_data, world_timestamps_data, output_folder)
    except Exception as e:
        error_message = f"An error occurred: {e}"
        print(error_message)
        log_error(subj, error_message)
    
    average_gaze_csv = os.path.join(output_folder, 'smartphone_averaged_gaze_data.csv')
    if not os.path.isfile(average_gaze_csv):
        break
    
    try:
        process_aoi_data(rectangle_data, world_timestamps_data, output_folder, pos_diff_threshold=0.05, rolling_frame_count=3)
    except Exception as e:
        error_message = f"An Error occurred: {e}"
        print(error_message)
        log_error(subj, error_message)

    aoi_frames = os.path.join(output_folder, 'smartphone_processed_aoi_for_each_frame.csv')

    if not os.path.isfile(aoi_frames):
        break
    try:
        process_gaze_data(video_path = subj_video, gaze_data_path=average_gaze_csv, rectangle_data_path=aoi_frames, output_folder=output_folder, scale_factor=1.0)
    except Exception as e:
        error_message = f"An error occurred: {e}"
        print(error_message)
        log_error(subj, error_message)

# Gaze Analysis

In [ ]:
test_subj_data = glob.glob(os.path.join(t7sheild,'et', 'sub-096_phone-*'))[0]

In [44]:
aoi_proportions_df

,event_id,proportion_within_aoi
0,5.0,0.000000
1,6.0,0.331361
2,7.0,0.314286


In [54]:
aoi_proportions = []

# we have to figure out how to deal with subjs 104-119
for folder in subj_folders[:]:
    subj = folder.split('/')[5].split('_')[0]
    cond = folder.split('/')[5].split('-')[2]

    # load in data from each CSV
    print('Loading data for ', subj, '...', 'phone-', cond)

    try:
        events_df = pd.read_csv(os.path.join(folder, 'events.csv'))
        updated_gaze_data_df = pd.read_csv(os.path.join(folder, 'smartphone_aois', 'updated_gaze_data.csv'))
    except Exception as e:
        print(f"An error occurred: {e}") 
        continue
    
    # Extract the recording.begin timestamp from events.csv
    recording_start_timestamp = events_df.loc[events_df['name'] == 'recording.begin', 'timestamp [ns]'].iloc[0]

    if cond == 'a':
        yt_cond = 'a'
        fb_cond = 'b' #onset of fb eeg event corresponds to start of fb screen recording, but fb et recording has it's own timestamp

    elif cond == 'b':
        yt_cond = 'b'
        fb_cond = 'a'

    try:

        print('starting youtube event extraction...')

        recording_start_yt_timestamp = events_df.loc[events_df['name'] == f'yt-{yt_cond}.start', 'timestamp [ns]'].iloc[0]
        recording_end_yt_timestamp = events_df.loc[events_df['name'] == f'yt-{yt_cond}.end', 'timestamp [ns]'].iloc[0]

        print(recording_start_yt_timestamp, recording_end_yt_timestamp)
        
        # Construct both potential filenames
        filename_variant_1 = f"sub_{subj[-3:]}_task_events.csv"
        filename_variant_2 = f"{subj}_task-events.csv"
        
        # Full paths for both filenames
        full_path_variant_1 = os.path.join(t7sheild, 'eeg', '{}_yt-phone'.format(subj), filename_variant_1)
        full_path_variant_2 = os.path.join(t7sheild, 'eeg', '{}_yt-phone'.format(subj), filename_variant_2)
        
        # Check if the first variant exists
        if os.path.exists(full_path_variant_1):
            eeg_events_yt_path  = full_path_variant_1
        # Check if the second variant exists
        elif os.path.exists(full_path_variant_2):
            eeg_events_yt_path = full_path_variant_2

        else:
            print(f"EEG events file not found for {subj} yt-{yt_cond} condition")
            continue
        
        eeg_events_yt = pd.read_csv(eeg_events_yt_path)
        eeg_events_yt['onset_ns'] = recording_start_yt_timestamp + (eeg_events_yt['onset'] * (1e9 / 250))
        eeg_events_yt['duration_ns'] = eeg_events_yt['duration'] * (1e9 / 250)
        # print(eeg_events_yt)

         # Iterate through each event in eeg_events_yt
        for _, event in eeg_events_yt.iterrows():
            onset_ns = event['onset_ns']
            duration_ns = event['duration_ns']
            end_ns = onset_ns + duration_ns
            
            # Subset updated_gaze_data_df for the current event's time range
            subset_gaze_data = updated_gaze_data_df[
                (updated_gaze_data_df['world_timestamp [ns]'] >= onset_ns) &
                (updated_gaze_data_df['world_timestamp [ns]'] <= end_ns)
            ]
            
            # Process the subsetted data as needed
            print(f"Processing event_id: {event['event_id']}")
            # print(subset_gaze_data)


            #Calculate the proportion of True within_aoi
            if not subset_gaze_data.empty:
                proportion_within_aoi = subset_gaze_data['within_aoi'].mean()
            else:
                proportion_within_aoi = 0
            
            # Store the result
            aoi_proportions.append({
                'subj': subj,
                'event_id': event['event_id'],
                'cond': 'yt',
                'proportion_within_aoi': proportion_within_aoi
            })
            
            # Process the subsetted data as needed
            # print(f"Processing event_id: {event['event_id']}")
            # print(subset_gaze_data)
            print(f"Proportion within AOI: {proportion_within_aoi}")

        # Convert the results to a DataFrame
            
    except Exception as e:
        print(f"An error occurred: {e}") 
        continue

    try:

        print('starting facebook event extraction...')

        recording_start_fb_timestamp = events_df.loc[events_df['name'] == f'fb-{fb_cond}.start', 'timestamp [ns]'].iloc[0]
        recording_end_fb_timestamp = events_df.loc[events_df['name'] == f'fb-{fb_cond}.end', 'timestamp [ns]'].iloc[0]

        print(recording_start_yt_timestamp, recording_end_yt_timestamp)
        
        # Construct both potential filenames
        filename_variant_1 = f"sub_{subj[-3:]}_task_events.csv"
        filename_variant_2 = f"{subj}_task-events.csv"
        
        # Full paths for both filenames
        full_path_variant_1 = os.path.join(t7sheild, 'eeg', '{}_fb-phone'.format(subj), filename_variant_1)
        full_path_variant_2 = os.path.join(t7sheild, 'eeg', '{}_fb-phone'.format(subj), filename_variant_2)
        
        # Check if the first variant exists
        if os.path.exists(full_path_variant_1):
            eeg_events_fb_path  = full_path_variant_1
        # Check if the second variant exists
        elif os.path.exists(full_path_variant_2):
            eeg_events_fb_path = full_path_variant_2

        else:
            print(f"EEG events file not found for {subj} fb-{fb_cond} condition")
            continue
        
        eeg_events_fb = pd.read_csv(eeg_events_fb_path)
        eeg_events_fb['onset_ns'] = recording_start_fb_timestamp + (eeg_events_fb['onset'] * (1e9 / 250))
        eeg_events_fb['duration_ns'] = eeg_events_fb['duration'] * (1e9 / 250)
        # print(eeg_events_fb)

         # Iterate through each event in eeg_events_yt
        for _, event in eeg_events_fb.iterrows():
            onset_ns = event['onset_ns']
            duration_ns = event['duration_ns']
            end_ns = onset_ns + duration_ns
            
            # Subset updated_gaze_data_df for the current event's time range
            subset_gaze_data = updated_gaze_data_df[
                (updated_gaze_data_df['world_timestamp [ns]'] >= onset_ns) &
                (updated_gaze_data_df['world_timestamp [ns]'] <= end_ns)
            ]
            
            # Process the subsetted data as needed
            print(f"Processing event_id: {event['event_id']}")
            # print(subset_gaze_data)


            #Calculate the proportion of True within_aoi
            if not subset_gaze_data.empty:
                proportion_within_aoi = subset_gaze_data['within_aoi'].mean()
            else:
                proportion_within_aoi = 0
            
            # Store the result
            aoi_proportions.append({
                'subj': subj,
                'event_id': event['event_id'],
                'cond': 'fb',
                'proportion_within_aoi': proportion_within_aoi
            })
            
            # Process the subsetted data as needed
            # print(f"Processing event_id: {event['event_id']}")
            # print(subset_gaze_data)
            print(f"Proportion within AOI: {proportion_within_aoi}")

        # Convert the results to a DataFrame
            
    except Exception as e:
        print(f"An error occurred: {e}") 
        continue

aoi_proportions_df = pd.DataFrame(aoi_proportions)
# print(aoi_proportions_df)

Loading data for  sub-095 ... phone- a
starting youtube event extraction...
1714570420459344000 1714570876660707000
Processing event_id: 1.0
Proportion within AOI: 0.24
Processing event_id: 2.0
Proportion within AOI: 0.576
Processing event_id: 3.0
Proportion within AOI: 0.176
Processing event_id: 4.0
Proportion within AOI: 0.336
starting facebook event extraction...
1714570420459344000 1714570876660707000
EEG events file not found for sub-095 fb-b condition
Loading data for  sub-096 ... phone- b
starting youtube event extraction...
1714584009088989000 1714584926954591000
Processing event_id: 5.0
Proportion within AOI: 0.0
Processing event_id: 6.0
Proportion within AOI: 0.33136094674556216
Processing event_id: 7.0
Proportion within AOI: 0.3142857142857143
starting facebook event extraction...
1714584009088989000 1714584926954591000
Processing event_id: 1.0
Proportion within AOI: 0.4666666666666667
Processing event_id: 2.0
Proportion within AOI: 0.0
Processing event_id: 3.0
Proportion wi

In [55]:
aoi_proportions_df.to_csv(os.path.join(t7sheild, 'average_ratios_phone.csv'), index=False)

In [56]:
average_proportions = aoi_proportions_df.groupby(['cond', 'event_id'])['proportion_within_aoi'].mean().reset_index()
average_proportions.to_csv(os.path.join(t7sheild, 'average_proportions.csv'), index=False)

In [ ]:
average_ratios[["fb_ratio", "yt_ratio"]] = average_ratios[["fb_ratio", "yt_ratio"]].apply(pd.to_numeric, errors='coerce')
average_ratios.dtypes
average_ratios['fb_ratio'].mean()

In [ ]:
platforms = ['Facebook Ads', 'YouTube Ads']
average_ratios_mean = [average_ratios['fb_ratio'].mean(), average_ratios['yt_ratio'].mean()]
print(average_ratios_mean)
sns.set(style='whitegrid', context='talk')
plt.figure(figsize=(6, 5))
plt.bar(platforms, average_ratios_mean, color=['blue', 'red'], alpha=0.7)
plt.xlabel('')
plt.ylabel('Average Screen On/Off Ratio')
plt.ylim(0.1, 0.45)
plt.grid(False)
plt.show()

# YT: RT to Skip and Number of Skips


# Extract Video of Scrolling Events